In [104]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
!pip install flask-ngrok 
!pip install python_speech_features
from python_speech_features.base import mfcc

In [106]:
import tensorflow.keras.models as models
from flask_ngrok import run_with_ngrok
from flask import Flask,jsonify,request
from sklearn.preprocessing import MinMaxScaler

In [107]:
from os import listdir
directory= "/content/drive/My Drive/Respiratory_Sound_Database/audio_and_txt_files/"
def ls(ruta = '.'):
    return listdir(ruta)
archivos=ls(directory)#lista de todos los archivos en la carpeta
archivos=[archivos[i][0:-4] for i in range(len(archivos)) if '.txt' in archivos[i]] 

diagnosis=pd.read_csv("/content/drive/My Drive/Respiratory_Sound_Database/patient_diagnosis.csv",header=None,names=['paciente','diagnostico'])
d=pd.get_dummies(diagnosis, columns = ['diagnostico'],sparse=False)

In [108]:
#Datos para Normalizar
path='/content/drive/MyDrive/Proyecto DBIO/Caracteristicas'
f=pd.read_csv(path+'/Caracteristicas_MFCC_50ms.csv',index_col=0)
data=f.values
X = data[:,:-3]
Y = data[:,-3:-1] #target para c y w
Y_copd= data[:,-1] #target para epoc
#scaler = StandardScaler()
scaler =MinMaxScaler((-1,1),True)#Nomaliza entre -1 y 1
X=scaler.fit_transform(X)

In [109]:
import librosa as lb
from scipy.signal import resample
def respiratory_cycle(s,resampl=0):
  '''
  Esta función extrae los ciclos respiratorios.
  Parametros
    s: nombre del archivo a leer (sin .wav)
    resampl: frecuencia a la que se quiere convertir
  Devuelve 
    data: conjunto de datos con los ciclos
    rate: frecuencia de muestreo de salida
    df_annot: conjunto de anotaciones
  '''
  path=directory + s +'.wav'
  x, rate = lb.load(path)

  path=directory + s +'.txt'
  df_annot = pd.read_csv(path,sep="\t")
  df_annot=df_annot.values

  if resampl!=0:
    x=resample(x,int(len(x)*(resampl/rate)),axis=0)
    rate=resampl

  data=[]

  #separar los ciclos
  for i in range(np.shape(df_annot)[0]):
    y=x[int(df_annot[i,0]*rate):int(df_annot[i,1]*rate)]
    y=y/np.max(np.abs(y))#normaliza

    data.append(y)
  return data,rate,df_annot

def cycle_characteristics(s,rate,window):
  m=mfcc(s, samplerate=rate, winlen=window, winstep=window, numcep=12, nfft=int(rate*window))
  m=np.concatenate((np.mean(m,axis=0),np.std(m,axis=0)))
  return [m]

In [110]:
model=models.load_model('/content/drive/MyDrive/Proyecto DBIO/Mejor_MFCC.h5')

In [111]:
f=archivos[33]
data,rate,df_annot=respiratory_cycle(f,resampl=4000)
entrada=np.array(cycle_characteristics(data[0],rate,0.05))

diagnostico=d[d['paciente']==int(f[0:3])]
diagnostico=diagnostico['diagnostico_COPD'].values

In [112]:
diagnostico[0]

1

In [113]:
entrada=scaler.transform(entrada)
entrada

array([[-0.0453095 ,  0.05709281,  0.41784993,  0.20485218,  0.01940341,
         0.01493783,  0.68470879,  0.10687189, -0.08449144, -0.02501745,
        -0.23609488, -0.16190569, -0.662357  , -0.71018014, -0.55071335,
        -0.50157211, -0.47079972, -0.32044745, -0.28306118, -0.36072764,
        -0.30390177, -0.42011726, -0.44380666, -0.45507903]])

In [114]:
y_pred=model.predict(entrada[:1,:])
y_pred.ravel()[0]

0.9999927

#Ejecución de la Aplicación

In [115]:
app = Flask(__name__)
run_with_ngrok(app) #empezar ngrok
global x

@app.route("/", methods = ['POST','GET','PUT'])
def home():
  global x
  text='a'
  if request.method == 'POST':
    x=request.form
    text='b'
    #print(x.to_dict())
    Datos=eval(list(x.to_dict().keys())[0])
    data=np.array(Datos["señal"],dtype=np.float32)
    print(data)

  diagnostico='No EPOC'
  if y_pred.ravel()[0]>0.8:
    diagnostico='Posible presencia de EPOC'
  return jsonify(diagnostico)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ba159a60bd05.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Nov/2020 20:38:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2020 20:38:37] "GET / HTTP/1.1" 200 -


[49. 13. 10. ... 10. 49. 48.]


...

127.0.0.1 - - [25/Nov/2020 22:00:49] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2020 22:00:49] "GET / HTTP/1.1" 200 -


[55. 13. 10. 53. 53. 13. 10. 53. 49. 13. 10. 52. 55. 13. 10. 52. 55. 13.
 10. 53. 54. 13. 10. 53. 53. 13. 10. 52. 55. 13. 10. 52. 54. 13. 10. 53.
 55. 13. 10. 53. 49. 13. 10. 52. 54. 13. 10. 52. 55. 13. 10. 53. 55. 13.
 10. 53. 52. 13. 10. 53. 49. 13. 10. 52. 57. 13. 10. 53. 54. 13. 10. 53.
 52. 13. 10. 52. 54. 13. 10. 52. 55. 13. 10. 53. 56. 13. 10. 53. 52. 13.
 10. 53. 49. 13. 10. 53. 50. 13. 10. 53. 57. 13. 10. 53. 51. 13. 10. 53.
 48. 13. 10. 52. 57. 13. 10. 53. 55. 13. 10. 53. 51. 13. 10. 52. 56. 13.
 10. 52. 57. 13. 10. 53. 57. 13. 10. 53. 54. 13. 10. 52. 57. 13. 10. 52.
 57. 13. 10. 53. 55. 13. 10. 53. 48. 13. 10. 52. 54. 13. 10. 52. 54. 13.
 10. 53. 52. 13. 10. 53. 49. 13. 10. 52. 55. 13. 10. 53. 48. 13. 10. 53.
 57. 13. 10. 53. 51. 13. 10. 52. 57. 13. 10. 52. 55. 13. 10. 53. 53. 13.
 10. 53. 49. 13. 10. 52. 52. 13. 10. 52. 54. 13. 10. 53. 54. 13. 10. 53.
 50. 13. 10. 52. 56. 13. 10. 53. 49. 13. 10. 53. 57. 13. 10. 53. 52. 13.
 10. 52. 56. 13. 10. 52. 54. 13. 10. 53. 54. 13. 10

127.0.0.1 - - [25/Nov/2020 23:07:40] "POST / HTTP/1.1" 200 -


[48. 13. 10. ... 13. 10. 48.]


127.0.0.1 - - [25/Nov/2020 23:07:53] "GET / HTTP/1.1" 200 -


In [116]:
Datos=eval(list(x.to_dict().keys())[0])

data=np.array(Datos["señal"],dtype=np.float32)
data=data/np.max(data)
data

array([0.84210527, 0.22807017, 0.1754386 , ..., 0.22807017, 0.1754386 ,
       0.84210527], dtype=float32)

In [117]:
len(data)/5

5822.2